<center>
<img src="https://upload.wikimedia.org/wikipedia/fr/thumb/1/1d/Logo_T%C3%A9l%C3%A9com_SudParis.svg/1014px-Logo_T%C3%A9l%C3%A9com_SudParis.svg.png" width="30%"></img>
</center>

<center> <h2> NET 4103/7431 Complex Network </h2> </center>

<center> <h3> Vincent Gauthier (vincent.gauthier@telecom-sudparis.eu) </h3> </center>

### Note
Avant de commencer les exercices, assurez-vous que tout fonctionne comme prévu. Tout d'abord, le redémarrage du kernel **(dans la barre de menus, sélectionnez le kernel $\rightarrow$ Restart)**.

Assurez-vous que vous remplir les célluler aux endroits marquer «YOUR CODE HERE». 

Veuillez supprimer les ligne «raise NotImplementedError()» dans toutes les cellules auxquelles vous avez répondu, ainsi que votre nom et prénom ci-dessous:

In [ ]:
NOM = "XXX"
PRENOM = "XXX"

---

<h1 align="center">TP 2: L'algorithme du PageRank</h3> 
<br />
<br />
<br />
<img src="../../images/webmap.jpg"></img>

# Introduction

Before 1998, the web graph was a largely unused source of information and ignored by search engines such as Yahoo, or Astalavista. However, researchers such as Jon Kleinberg [2] and Brin and Page [3] have used this resource to create an algorithmically elegant methodology that has helped develop the search engines we know today. They then mainly used the following idea: a hyperlink link from my web page to another page can be interpreted as a recommendation from me to the web page in question. The underlying idea that is exploited is the following: a web page that is quoted very often must have a better ranking than a less frequently cited web page. However, this idea is not new, but used as the sole selective criterion is not enough. For example, a letter of recommendation for a job from the CEO of Orange will surely carry more weight than the other 10 letters of recommendation that you could have received from random people. In conclusion, the importance of the recommendations (and not only their numbers) must also be taken into account in calculating the reputation of a web page. This is exactly the principle of **PageRank**.

In current search engines, each indexed webpage is associated with a "**PageRank**" value. When you perform a search, the search engine returns the web pages corresponding to the keywords sorted in ascending order of their value of "PageRank". The underlying assumption that is used is that: the web page with the highest "PageRank" value must be the most relevant for the keywords considered.

In short, the notoriety (PageRank value) of a web page is defined as follows: the reputation of a web page is important, if it is itself pointed by other web pages with significant notoriety. This circular reasoning is at the heart of the algorithm that will be developed during this exercise. Through this circular reasoning, we deduce that the values of PageRank are in fact the values of the stationary states of an immense Markov chain. The transitions of this Markov chain are defined by the web graph. The matrix of transitions of this Markov chain is called "Google" matrix $\mathbf{G}$. In order to calculate this stationary state vector, however, the Markov chain in question must have a unique solution. For this, one of the conditions is that the transitions graph must be irreducible (ie the graph forms a connected component), which is not the case for the web graph (cf. Fig. 1). The PageRank algorithm allowed Brin and Page to intelligently bypass this difficulty applying a transformation on the adjacency matrix in order to assume the irreducibility of this one.

<div align="center">
<img src="../../images/Scc.png" width="500px"></img>
** Figure 1**
</div>

**PageRank's Algorithm**
1. Normalization of the adjacency matrix of the web graph to generate a stochastic matrix
2. we update the matrix from the previous step in order to take into account the Dangling nodes (the node that don't have any outgoing link)
3. Create the "Google" Matrix
4. Compute the stationary state of the Markov chain

In the first part of this exiercice we are going to considere the follwing directed graph $\mathcal{G}(\mathcal{V}, \mathcal{E})$ (cf. **Fig. 2**), as small example of the web graph. Each edge $e \in \mathcal{E}$ represente one hyperlink from one webpage $v \in \mathcal{V}$ to another.

<div align="center">
<img src="../../images/GraphPageRank.png" width="400px"></img>
**Figure 2**
</div>

### Notations
$\mathbf{A}$: The adjacency matrix of the graph $\mathcal{G}(\mathcal{V}, \mathcal{E})$

$\mathbf{H}$: Normalized adjacency matrix

$\mathbf{G}$: The dense, stochastic matrix called teh "Google" matrix

$\mathbf{\pi}^T$: PageRank Vector, a stationnay vector of the Markov chain describe by the matrix $\mathbf{G}$

$n$ : Number of the web page (in our graph)

$\alpha$: parameter between 0 and 1

### Conventions

$\mathbf{e}^T$ : is vector line $\begin{pmatrix} 1 & 1 & 1 & 1 \end{pmatrix}$

$\mathbf{e}$ : is a vector colomn  $\begin{pmatrix} 1 \\ 1 \\ 1 \\ 1 \end{pmatrix}$

$\sigma(\mathbf{A})$ est l'ensemble (set) des valeurs propres de la matrice $\mathbf{A}$

$\lambda_{max} = \max_{\lambda \in \sigma(\mathbf{A})} \lvert \lambda \lvert$ est la plus grande valeur propre de la matrice $\mathbf{A}$

$\mathbf{e} \otimes \mathbf{e}^T$ : est le produit exterieur $\begin{pmatrix} 1 \\ 2 \\ 3  \end{pmatrix}  \otimes \begin{pmatrix} 1 & 2 & 3 \end{pmatrix} = \begin{pmatrix} 1 & 2 & 3  \\ 2 & 4 & 6 \\ 3 & 6 & 9 \end{pmatrix}$

### Python Reminder 
Multiplication and division with Numpy:
```Python
import numpy as np
x = np.array([2, 2, 2])
2*x
>>> array([4, 4, 4])
x = np.array([2, 2, 2])
x/2
>>> array([1, 1, 1])
A = np.array([[2, 2, 2], [2, 2, 2], [2, 2, 2]])
A/2
>>> np.array([[1, 1, 1], 
              [1, 1, 1], 
              [1, 1, 1]])
```

Matrix vector division:
```Python
A = np.array([[1, 1], [4, 4]])
b = np.array([1, 3])
(A.T/b).T
>>> array([[ 1.        ,  1.        ],
           [ 1.33333333,  1.33333333]])
```

Dot product:
```Python
import numpy as np

A = np.array([[2, 2, 2], [2, 2, 2]])
x = np.array([2, 2, 2])
np.dot(A,x)
>>> array([12, 12])

a = np.array([1, 1, 1])
b = np.array([1, 1, 1])
np.outer(a,b)
>>> array([[1, 1, 1], 
           [1, 1, 1], 
           [1, 1, 1]])
```

Matrix transpose:
```Python
import numpy as np

A = np.array([[1, 2], [3, 4]])
A.T
>>> array([[1, 3],
           [2, 4]])
```

Shape of a matrix
```Python
A = np.array([[1, 1, 1], [1, 1, 1]])
n, m = A.shape
print(n, m)
>>> 2 3
```

Row sum of a Matrix
```Python
A = np.array([[1, 1, 1,], [2, 2, 2]])
A[1,:].sum()
>>> 6
```

In [ ]:
import networkx as nx
import numpy as np
import scipy as sp
from numpy import linalg as LA
import pandas as pd
import csv
import matplotlib.pylab as plt

%matplotlib inline

# Style pour le Notebook
from IPython.core.display import HTML

def css_styling():
    styles = open("../../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# Generate a synthetic web-graph with networkx

With the help of networkx generate the directed graph $\mathcal{G}(\mathcal{V}, \mathcal{E})$ (cf. **Fig 2**), and draw the graph with matplotlib in order to validate visualy that you obtain the same graph as in the **figure 2**. 

**Help**: Use the already defined functions in the networkx library to add vertices and edges to the graph ([networkx tutorial](https://networkx.github.io/documentation/networkx-1.10/tutorial/tutorial.html#nodes))

In [ ]:
plt.figure()
g = nx.DiGraph()

# YOUR CODE HERE
raise NotImplementedError()

pos = nx.spring_layout(g);
nx.draw_networkx(g, pos=pos, node_size=600, font_size=20.0);
plt.axis('off');

In [ ]:
assert len(g.nodes()) == 6
assert g.has_edge(6,4)
assert g.has_edge(4,6)
assert g.has_edge(2,5) == False

# Extract the Adjacency matrix of the graph 

Before going further in the exercise, let's extract the adjacency matrix $\mathbf{A}_{n \times n}$ from the directed graph $\mathcal{G}(\mathcal{V}, \mathcal{E})$ to construct the normalized adjacency matrix of the $\mathbf{H}_{n \times n} $ graph.

(eq. **1**)
$$
\mathbf{H}_{ij} = \begin{cases}
    1/\sum_{j} \mathbf{A}_{ij} & \text{ if there is a link between node } i \text{ and } j\\
    0 & \text{ otherwise}
\end{cases}
$$

**Exemple**: The matrix $\mathbf{H}_{n \times n}$ of the graph in figure 2.


$$
\mathbf{H} = \begin{pmatrix}
  0 & 1/2 & 1/2 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 \\
  1/3 & 1/3 & 0 & 0 & 1/3 & 0 \\
  0 & 0 & 0 & 0 & 1/2 & 1/2 \\
  0 & 0 & 0 & 1/2 & 0 & 1/2 \\
  0 & 0 & 0 & 1 & 0 & 0 \\
\end{pmatrix}
$$

**Example: How to export the adjacency matrix from a networkx graph**
```Python
>>> import numpy as np
>>> G = nx.Graph([(1,1)])
>>> A = nx.to_numpy_matrix(G)
>>> A
matrix([[ 1.]])
```

In [ ]:
def transform_to_stochatic(G):
    ### Ignore division by zero warmings
    np.seterr(divide='ignore', invalid='ignore')
    
    # YOUR CODE HERE
    raise NotImplementedError()
    return S

In [ ]:
H = transform_to_stochatic(g)

H_res = np.array( 
[[ 0.,          0.5,         0.5,         0.,          0.,          0.,        ],
 [ 0.,          0.,          0.,          0.,          0.,          0.,        ],
 [ 0.33333333,  0.33333333,  0.,          0.,          0.33333333,  0.,        ],
 [ 0.,          0.,          0.,          0.,          0.5,         0.5,       ],
 [ 0.,          0.,          0.,          0.5,         0.,          0.5,       ],
 [ 0.,          0.,          0.,          1.,          0.,          0.,        ]])

np.testing.assert_allclose(H, H_res)

# finding dangling nodes 

We want to calculate the "dangling vector" $\mathbf{a}$ which associates with each node the value 1 if this one does not have any outgoing link, and 0 otherwise.

(eq. **2**)

$$
\mathbf{a}_i = \begin{cases}
    1 & si \sum_{j} \mathbf{A}_{ij} = 0 \\
    0              & \text{ otherwise}
\end{cases}
$$

**Example**:
<img src="../../Images/DanglingPageRank.png" width="500px"></img>
<div align="center"><b>Figure 3</b>: the correponding vector $\mathbf{a}$ of the graph defined in figure 2.</div>

In [ ]:
def dangling_vector(H):
    # YOUR CODE HERE
    raise NotImplementedError()
    return a

In [ ]:
a = dangling_vector(H)
np.testing.assert_array_equal(a, [ 0.,  1.,  0.,  0.,  0.,  0.])

# Create the "Google" Matrix 

Unfortunately, the graph of the web does not form a connected graph, which is neither irreducible nor aperiodic. By constructing the "Google" matrix, we will add artificial transitions to the adjacency matrix of the web graph in order to transform it into another graph that offers satisfactory properties (irreducible and aperiodic). Once the $ \mathbf{G}_{n \times n}$ matrix has been built, it will be used to calculate the steady states $\pi^T$ (the PageRank) of each node of the graph $\mathcal{G}(\mathcal{V}, \mathcal{E})$. The goal of this exercise is to transform the $\mathbf{A}_{n \times n}$ adjacency matrix of the web graph into a matrix $\mathbf{G}_{n \times n}$ so that it satisfies the following conditions:

1. The matrix $\mathbf{G}$ should be stochastic. 
2. The matrix $\mathbf{G}$ should be irreductible.
3. The matrix $\mathbf{G}$ should be aperiodic.
4. The matrix $\mathbf{G}$ should be primitive. 

<div class=warn>
<b>Definition:</b> A matrix is called primitive if it admits a power whose terms are strictly positive (i.e., there is $ \mathbf{G}^k > 0 $).
</div>

<div class=warn>
<b>Theorem:</b> if the matrix $\mathbf{G}$ is primitive the it exist a vector $\pi^{(k)T}$ such that $\mathbf{\pi}^{(k)T} = \mathbf{\pi}^{(k)T} \mathbf{G}$. In other words, there is convergence of the vector $\pi^{(k)}$ to a single stationary state when $k \to \infty$, see Perron-Frobenius theorem.
</div>

<div class=warn>
<b>Theorem of Perron-Frobenius:</b> Let $ \mathbf{A} $ be a positive and primitive matrix. Then there exist a value $ \lambda_{max}$ such that:

1. $\lambda_{max}$ is a real positive value, $\lambda_{max} > 0$ <br>
2. $\lambda_{max}$ is associated to eigenvectors strictly positive <br>
3. $\lambda_{max} > \lvert \lambda \lvert\ \ \forall \lambda \neq \lambda_{max}$ <br>
4. it exist a unique vector $\mathbf{x}$ (with $\lvert\lvert \mathbf{x} \lvert\lvert_{1} = 1$) such that  $\mathbf{A}\mathbf{x}=\lambda_{max} \mathbf{x}$
</div>

<div class=green>
<b>Conclusion:</b> the adjacency matrix $\mathbf{A}_{n \times n}$ of an ireductible and aperiodic graph is a primitve matrix. Alors d'après le thèoreme de Perron-Frobenius on peut en conclure que la matrice d'adjacence $\mathbf{A}_{n \times n}$ admet un unique vecteur $\mathbf{x}$ de norme 1 à coordonnées strictement positives tel que $\mathbf{A}\mathbf{x}=\lambda_{max} \mathbf{x}$.
</div>

On pose :

(eq. **3**)
$$
\mathbf{S} = \mathbf{H} + \frac{1}{n} \mathbf{a} \otimes \mathbf{e}^T \\
$$

Pour rappel, le vecteur $\mathbf{a}_n$ (calculé à l'étape 3. de l'exercice) est le vecteur Dangling et $\mathbf{H}_{n \times n}$ est la matrice normalisée d'adjacence du graphe orienté $\mathcal{G}(\mathcal{V}, \mathcal{E})$ (calculé à l'étape 2. de l'exercice). On definit la matrice alors la "Google matrice" comme étant: 

(eq. **4**)
$$
\mathbf{G} = \alpha  \mathbf{S} + (1-\alpha) \frac{1}{n}\mathbf{e} \otimes \mathbf{e}^T \\
$$



<img src="../../images/GoogleMatrice.png" width="700px"></img>
<div align="center">**Fig. 4**: Google matrice</div>

La "Google matrice" $\mathbf{G}_{n \times n} $ (cf. eq. 4) est construite en ajoutant des transitions supplémentaires (cf. Fig. 3) à la matrice d'adjacence originale du graphe $\mathcal{G}(\mathcal{V}, \mathcal{E})$ de telle sorte que les conditions 1 à 3 soient satisfaites.

**Question 1**: Développé l'équation 4 (à l'aide de l'équation 3) afin d'exprimer la matrice $\mathbf{G}_{n \times n}$ en fonction de $\mathbf{H}_{n \times n}$ et $\mathbf{a}_n$


**reponse**:



#### Implémenter la fonction qui construit la "Google matrice"  (reponse de la question 1)

In [ ]:
def google_matrix(H, a, α=0.9):
    '''
    H: matrix d'adjacence stochastique
    a : Dangling vector
    '''
    # YOUR CODE HERE
    raise NotImplementedError()
    return G

In [ ]:
G = google_matrix(H, a)
print(G)

G_res = np.array(
[[ 0.01666667,  0.46666667,  0.46666667,  0.01666667,  0.01666667,  0.01666667],
 [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667],
 [ 0.31666667,  0.31666667,  0.01666667,  0.01666667,  0.31666667,  0.01666667],
 [ 0.01666667,  0.01666667,  0.01666667,  0.01666667,  0.46666667,  0.46666667],
 [ 0.01666667,  0.01666667,  0.01666667,  0.46666667,  0.01666667,  0.46666667],
 [ 0.01666667,  0.01666667,  0.01666667,  0.91666667,  0.01666667,  0.01666667]])

np.testing.assert_allclose(G, G_res, rtol=1e-5)

#### Application numerique pour le graphe décrit dans la figure 2 ($\alpha = 0.9$)

(**eq. 4.**)
$$
\mathbf{G} = 0.9\mathbf{H} + \left[ 0.9 \begin{pmatrix} 0 \\ 1 \\ 0 \\ 0 \\ 0 \\ 0 \\ \end{pmatrix} + 0.1  \begin{pmatrix} 1 \\ 1 \\ 1 \\ 1 \\ 1 \\ 1 \\ \end{pmatrix}  \right]  \otimes 1/6  \begin{pmatrix} 1 & 1 & 1 & 1 & 1 & 1 \end{pmatrix} \\
$$

$$
\mathbf{G} =  \begin{pmatrix} 
1/60 & 7/15 & 7/15 & 1/60 & 1/60 & 1/60 \\
1/6  & 1/6  & 1/6  & 1/6  & 1/6  & 1/6 \\
19/60 & 19/60 & 1/60 & 1/60 & 19/60 & 1/60 \\
1/60 & 1/60 & 1/60 & 1/60 & 7/15 & 7/15 \\
1/60 & 1/60 & 1/60 & 7/15 & 1/60 & 7/15 \\
1/60 & 1/60 & 1/60 & 11/12 & 1/60 & 1/60 \\
\end{pmatrix}
$$

# Calculer la valeur du vecteur PageRank

(**eq. 5**)
\begin{eqnarray}
& \mathbf{\pi}^{(k)T} & =   \mathbf{\pi}^{(k-1)T} \mathbf{G} \\ 
& \mathbf{\pi}^{(k-1)T} & = \mathbf{\pi}^{(k-2)T} \mathbf{G} \\ 
& \vdots &\\
& \mathbf{\pi}^{(1)T} & = \mathbf{\pi}^{(0)T} \mathbf{G} 
\end{eqnarray}

Par récurence on peut donc écrire 

(**eq. 6**)
$$\mathbf{\pi}^{(k)T} = \mathbf{\pi}^{(0)T} \mathbf{G}^k$$

On cherche maintenant à calculé le vecteur stationnaire $\mathbf{\pi}^{(k)T}$ lorsque $k \to \infty$. On sait que la matrice $\mathbf{G}$ est primitive par construction, alors la converge du vecteur $\mathbf{\pi}^{T}$(dans le cas d'un espace d'état fini) est prouvé par le théorème de Perron-Frobenius. Afin de calculer les états stationaires du vecteur $\mathbf{\pi}^{T}$ on va alors itérer l'opération $\mathbf{\pi}^{(k)T} = \mathbf{\pi}^{(k-1)T} \mathbf{G}$ jusqu'a convergence du vecteur $\mathbf{\pi}^{T}$ (cf. Alg 1).

<br>

<img src="../../images/PowerIt.png" width="600px"></img>

<br>
<div align="center">**Alg1** : Algorithme de calcule iteratif du PageRank</div>

In [ ]:
def power_iter(G, max_iter=500, tol=1e-6):
    """power_iter

    Parametres
    -----------
    G : Goole matrice

    max_iter : integer, optional
        Nombre maxium d'iteration permise 

    tol : float, optional
       Error tolerance used to check convergence in power method solver.
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    raise RuntimeError('pagerank: power iteration failed to converge in %d iterations.' % max_iter)

In [ ]:
(π, gap) = power_iter(G, max_iter=100, tol=1e-6)
print(π)

π_res = np.array([ 0.03721313,  0.05395937,  0.04150701,  0.37507848,  0.20599777,  0.28624425])

np.testing.assert_allclose(π, π_res, rtol=1e-5)

In [ ]:
λ, _ = LA.eig(G)
λ_2 = np.sort(λ.real)[-2]
λ_gap = [λ_2 ** i for i in range(1,25)]

fig = plt.figure(figsize=(7,4))
plt.semilogy(gap, 'ro')
plt.semilogy(λ_gap, lw=2.0, label="$\lambda_2^i$")
plt.title("Vitesse de convergence de l'algorithme du PageRank")
plt.xlabel("Nombre d'iterations de l'agorithme PageRank $(i)$")
plt.ylabel("iteration gap")
plt.legend()
plt.tight_layout()
print(π)

# 5. Intégration des trois étapes qui constituent l'algorithme du PageRank

Construire la fonction qui intégre les différentes étapes qui constitutent l'algorithme du PageRank. Vous allez utiliser les fonctions que vous avez définies précedemment pour construire l'algorithme final. 

**Algorithme du PageRank**
1. Normalisation de la matrice d'ajacence du graphe du web
2. Découverte des noeuds du graphe du web ne possédant pas de liens de sortie "Dangling nodes"(aucun lien hypertext)
3. Creation de la "Google" Matrice 
4. Calcul de la vecteur des états stationnaires de la chaine de Markov $\mathbf{G}$ 


In [ ]:
def pagerank(Graph, α=0.85, max_iter=100, tol=1e-6):
    """Return the PageRank of the nodes in the graph.

    PageRank computes a ranking of the nodes in the graph G based on
    the structure of the incoming links. It was originally designed as
    an algorithm to rank web pages.

    Parametres
    -----------
    G : graph
      A NetworkX graph.  Undirected graphs will be converted to a directed
      graph with two directed edges for each undirected edge.

    alpha : float, optional
      Damping parameter for PageRank, default=0.85.

    max_iter : integer, optional
      Maximum number of iterations in power method.

    tol : float, optional
      Error tolerance used to check convergence in power method solver.
      
    Return
    PR : dict
      retourne un dictionnaire avec comme clé le nodeid, et comme valeur le pagerank du noeud 
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return PR

In [ ]:
PR = pagerank(g)
print(PR)

PR_res = dict({
    1: 0.05170556259095016, 
    2: 0.07368068204240269, 
    3: 0.057413363969125462, 
    4: 0.3487020460725242, 
    5: 0.1999034157779406, 
    6: 0.2685949295470571})

for k,v in PR_res.items():
    np.testing.assert_allclose(PR[k], PR_res[k], rtol=1e-5)

# Application de l'algorithme du PageRank pour le classement des pages web

## Exemple avec le graphe des liens du site Wikipedia

In [ ]:
# On charge le graphe des liens des page wikipedia dans un graphe (networkx)
Gwikipedia = nx.read_graphml("../../Data/Wikipedia/wikipedia.graphml")

#On charge la base de données des pages dans un tableau Pandas
wikipedia_db = pd.read_pickle("../../Data/Wikipedia/wikipedia.db")
wikipedia_db.head()

## Calcule des propriété statistique du graphe les liens des pages wikipedia 

Nous allons calculer:
1. la densité du graphe (rappel la densité d'un graphe $\mathcal{G}(\mathcal{V}, \mathcal{E})$ est $D =\vert \mathcal{E}\vert/N(N-1)$) 
2. la densité de probabilité empirique du degré des noeuds du graphe 
3. le complémentaire de la fonction de répartition empirique du degré des noeuds du graphe

In [ ]:
#question 1

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#
# On calcule la distribution des degree du graphe des pages wikipedia question 2 et 3
#
degree = [v for k,v in dict(Gwikipedia.degree()).items()]
distribution = [(elem, degree.count(elem)) for elem in sorted(set(degree))]
k,pk = zip(*distribution)
PDF = np.array(pk)/sum(pk)
CCDF = 1-np.cumsum(PDF)


#
# Afichage 
#
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.loglog(k, PDF, 'ro')
ax1.set_xlabel("$k$ Degree")
ax1.set_ylabel("$P_k$")
ax1.set_title("PDF")

ax2.loglog(k, CCDF, 'ro')
ax2.set_ylim(1e-4,1.1)
ax2.set_xlim(1,2e3)
ax2.set_xlabel("$k$ Degree")
ax2.set_ylabel("$1-P[K > k]$")
ax2.set_title("CCDF")
fig.tight_layout()

** Question:**

- La matrice d'adjacence du graphe est-elle dense ? ou au contraire creuse ?
- Que dire de la distribution du degrée des articles wikipedia ?

## Calculer le PageRank du graphe des articles  Wikipedia

In [ ]:
# Calcul du PageRank en utilisant la fonction PageRank déja implementé

# YOUR CODE HERE
raise NotImplementedError()

PR = [[k,v] for k,v in PR.items()]
PR = pd.DataFrame(PR, columns=['PageID', 'PageRank'])
PR = PR.set_index('PageID')
PR

In [ ]:
# On associe un PageRank à chaque page wikipedia dans la base de données 
wikipedia_db = wikipedia_db.join(PR)
# On trie les entrées wikipedia par ordre croissant de leur PageRank
wikipedia_db = wikipedia_db.sort_values(['PageRank'], ascending=0)
# on affiche les premières entrées triées par ordre croissant 
wikipedia_db[["Page Title", "PageRank"]].head()

## Effectuer une recherche par mots clés sur la base de données des articles Wikipedia

**Attention**: le nombre de pages wikipedia indexées dans cette base de données est de l'ordre de 5000 articles seulement, rédigé en anglais. Le nombre d'articles et donc de mots clées sont donc restreints. Effectuer les requêtes avec des mots clés ecrit en lettres minuscules uniquements.

In [ ]:
# Exemple de mots cléss 
mot_clee_1 = "france"
mot_clee_2 = "germany"
# Requete sur la base de données par mots clés
wikipedia_db[(wikipedia_db['Keywords'].str.contains(mot_clee_1)==True) & (wikipedia_db['Keywords'].str.contains(mot_clee_2)==True)].head(10)

In [ ]:
# Exemple de mots clées 
mot_clee_1 = "france"
# Requete sur la base de données par mots clée
wikipedia_db[(wikipedia_db['Keywords'].str.contains(mot_clee_1)==True) ].head(10)

**Question**:

Effectuez conjointement: une requête sur la base de donnée et avec le moteurs de recherche de google: 

* Faite une Requete google "site:wikipedia.org mot_clée1 mot_clée2"
* Comparer qualitativement les requetes faite sur Google et celles effectués sur la base de données

# Implémentation du PageRank avec un algorithme distribué


Le classement des page web au travers de l'algorithme du "PageRank" est encore aujourd'hui considéré comme étant le plus gros problème matriciel connu a ce jour. Pour vous donner un ordre de grandeur, en 2007 la "power iteration" qui permetait le calcul du PageRank chez Google occupait un datacenter pendant 15 jours pour finaliser le calcul. Afin de reduire la complexité du calcul $\pi^T \mathbf{G}$ on souhaiterait effectuer un calcul sur une matrice creuse à la place de la matrice $\mathbf{G}$ qui est dense. Dans le même temps, on souhaite pouvoir paralléliser le calcul (multiprocesseur, cluster de calcul). Une manière élégante d'obtenir ces deux propriétés est d'éffectuer l'opération suivante sur tout les noeuds du graphe (Cf. eq. 7., Fig. 5, Algo. 3):
<br>
(**eq. 7**)
$$ PR_{i} = \frac{(1-\alpha)}{n} + \alpha \sum_{j \in \mathcal{N}(i)} \frac{PR_j}{L_j}$$
<br>
<br>
<div align="center">
<img src="../../images/PageRankDistributed2.png" width="400px"></img>
** Figure 5.**: PageRank 
</div>

L'opération decrite dans l'equation 7. peut être calculer de manière indépendante (en parallèle) pour tous les noeuds du graphes. C'est l'approche qui est développée dans des logiciels pour de calcul de grands volumes de données tels que: 

* [Spark/Graphx](http://spark.apache.org/)
* [Graphlab](https://dato.com/)

ou plus simplement dans certaines librairies de calcul parallèle utilisant la thèorie des graphes:

* [boost graph](http://www.boost.org/doc/libs/1_46_0/libs/graph_parallel/doc/html/page_rank.html)
* [graph-tool](https://graph-tool.skewed.de/)
<br>
<br>
<div align="center">
<img src="../../images/PageRankDistributed.png" width="500px"></img>
** Algo 3.**: Algorithme simplifié du PageRank
</div>

In [ ]:
def pagerank_distributed(g, α=0.9, max_iter=200, tol=1e-6):
    N = len(g.nodes())
    gc = g.copy()
    
    # on modifie le graphe pour ajouter les transitions au dangling node  
    for dangling in g.nodes():
        if gc.out_degree(dangling) == 0:
            for n in g.nodes():
                gc.add_edge(dangling, n)

    # on initialise le pagerank a chaque noeud du graphe
    for node in gc.nodes():
        gc.node[node]['PageRankOld'] = 1.0/N
    
    # Power iteration
    for _ in range(max_iter):
        # YOUR CODE HERE
        raise NotImplementedError()
        
        if tol > N*diff:
            return np.array([gc.node[n]['PageRank'] for n in gc.nodes()])

In [ ]:
PR = pagerank_distributed(g)
print(PR)

PR_res = np.array([
    0.0372119873811025, 
    0.05395738811699192, 
    0.04150567933532273, 
    0.37508077029418346, 
    0.20599832111968525, 
    0.28624585375271416])

np.testing.assert_allclose(PR, PR_res, rtol=1e-5)

## References

[1] Amy N. Langville, Carl D. Meyer, "[Deeper Inside PageRank](http://projecteuclid.org/euclid.im/1109190965)", Internet Math., Vol. 1(3), pp. 335--380, 2003.

[2] Jon Kleinberg, "[Authoritative sources in a hyperlinked environment](http://www.cs.cornell.edu/home/kleinber/auth.pdf)",  Journal of the ACM, 46(5), pp. 604–632, 1999.

[3] L. Page, S. Brin, R. Motwani, T. Winograd, "[The PageRank citation ranking: Bringing order to the Web](http://ilpubs.stanford.edu:8090/422/1/1999-66.pdf)", published as a technical report on January 29, 1998.